# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csvpath = "/Users/yuhsichen/Desktop/python-api-challenge/WeatherPy/output_weather_df.csv"
city_df = pd.read_csv(csvpath)
city_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Busselton,-33.6500,115.3333,56.57,68,0,12.59,AU,1669052369
1,1,Albany,42.6001,-73.9662,37.02,50,53,9.44,US,1669052268
2,2,Salto,-31.3833,-57.9667,66.36,94,100,9.22,UY,1669052110
3,3,Kapaa,22.0752,-159.3190,75.52,82,75,23.02,US,1669052370
4,4,Faanui,-16.4833,-151.7500,79.03,75,22,15.46,PF,1669052370
...,...,...,...,...,...,...,...,...,...,...
529,529,Rafai,4.9500,23.9167,75.60,45,62,2.64,CF,1669052292
530,530,Ambovombe,-25.1667,46.0833,66.81,65,51,6.85,MG,1669052543
531,531,Dwārka,22.2394,68.9678,78.51,58,1,6.91,IN,1669052544
532,532,Bonavista,48.6499,-53.1147,32.92,89,66,50.53,CA,1669052544


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
gmaps.configure(api_key=g_key)

locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"].astype(float)
cities = city_df["City"]

fig = gmaps.figure("TERRAIN")

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
#Max_temp between 96.8F to 65F
optimal_temp = city_df.loc[(city_df["Max Temp"] < 96.8) & (city_df["Max Temp"]>65)]

#Wind speed less than 4.5 m/s
optimal_windspeed = optimal_temp.loc[optimal_temp["Wind Speed"]<4.5]

#Zero cloudiness
final_weather_df = optimal_windspeed.loc[optimal_windspeed["Cloudiness"] <10]

reset_weather_df = final_weather_df.reset_index()

weather_df = reset_weather_df[["City","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Country","Date"]]
weather_df





,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Salalah,17.0151,54.0924,80.69,69,0,3.44,OM,1669052369
1,Isangel,-19.5500,169.2667,78.33,81,5,3.49,VU,1669052401
2,Sur,22.5667,59.5289,77.04,65,1,3.74,OM,1669052524
3,Saint-Pierre,-21.3393,55.4781,74.88,73,0,2.30,RE,1669052529


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df = []
weather_df["Hotel Name"] = ""

params = {
    "radius": 5000,
    "keyword": "hotel",
    "types": "lodging",
    "key": g_key

}

for index, row in weather_df.iterrows():
    location = f'''{row["Lat"]},{row["Lng"]}'''
    params["location"] = location

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_info = requests.get(url, params = params).json()
    hotel_df.append(hotel_info)


    try:
        print(f"The hotel name is: {hotel_info['results'][0]['name']}.")
        weather_df.loc[index, "Hotel Name"] = hotel_info["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")



weather_df







/var/folders/bk/gvxhl3kx4zv9t13q1slc_8h80000gn/T/ipykernel_39800/3949584110.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df["Hotel Name"] = ""


The hotel name is: Darbat Hotel.
The hotel name is: Volcano Sea View Tree House.
The hotel name is: Sur Grand Hotel.
The hotel name is: Villa Delisle.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Salalah,17.0151,54.0924,80.69,69,0,3.44,OM,1669052369,Darbat Hotel
1,Isangel,-19.5500,169.2667,78.33,81,5,3.49,VU,1669052401,Volcano Sea View Tree House
2,Sur,22.5667,59.5289,77.04,65,1,3.74,OM,1669052524,Sur Grand Hotel
3,Saint-Pierre,-21.3393,55.4781,74.88,73,0,2.30,RE,1669052529,Villa Delisle


In [14]:
hotel_df = weather_df

In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """

Hotel Name
      {Hotel Name}
City
  {City}
Country_Date
  {Country_Date} 

  
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Country_Date'

In [20]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='red',
    stroke_color='black', scale=2,
    info_box_content=hotel_info
)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig


Figure(layout=FigureLayout(height='420px'))